In [2]:
!pip install orjson


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import json
import os
import orjson
import glob
import sys
import time
from collections import defaultdict
from datetime import datetime,timedelta
import math
from collections import Counter
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [4]:
### Reading Data
df= pd.read_csv('s3://scien-ds-sm-root-backup/home/pradeep/22_Unified_LN_unsec_V2.0/Client_Data_Unsec/040424_LN_V2_Unsec_Dataset_With_Performance_Race_Age_and_Gender.csv')
df.shape

(306791, 342)

In [5]:
df[['approve_flag', 'funded_flag', 'bad_flag']].sum()

approve_flag    162190.0
funded_flag     148414.0
bad_flag          5010.0
dtype: float64

In [6]:
df['client'].value_counts(dropna= False)

client
Bethpage    69561
Guardian    53335
Numerica    47214
Arkansas    41008
Tower       40826
CPM         28398
Hawaii      11880
GESA         9671
HFCU         4898
Name: count, dtype: int64

In [7]:
df['crossindustry_score'].describe()

count    306503.000000
mean        667.319273
std         124.601294
min         100.000000
25%         645.000000
50%         688.000000
75%         736.000000
max         872.000000
Name: crossindustry_score, dtype: float64

In [8]:
df= df[df['crossindustry_score']>500].reset_index(drop= True)
df.shape

(290379, 342)

In [9]:
df['app_date']= df['app_date'].apply(lambda x: pd.to_datetime(x[:10]) if pd.isna(x)== False else x)
df['app_date'].describe()

count                           290379
mean     2019-11-08 23:44:13.518333184
min                2016-12-09 00:00:00
25%                2018-06-07 00:00:00
50%                2019-09-18 00:00:00
75%                2021-05-27 00:00:00
max                2022-09-29 00:00:00
Name: app_date, dtype: object

In [10]:
df.head(2)

,unique_id,client,app_date,approve_flag,funded_flag,fico_pb,fico_cb,bad_flag,product,channel,fico_cut,filename,seq,auto_score,auto_reason1,auto_reason2,auto_reason3,auto_reason4,auto_reason5,bankcard_score,bankcard_reason1,bankcard_reason2,bankcard_reason3,bankcard_reason4,bankcard_reason5,short_term_lending_score,short_term_lending_reason1,short_term_lending_reason2,short_term_lending_reason3,short_term_lending_reason4,short_term_lending_reason5,telecommunications_score,telecommunications_reason1,telecommunications_reason2,telecommunications_reason3,telecommunications_reason4,telecommunications_reason5,crossindustry_score,crossindustry_reason1,crossindustry_reason2,crossindustry_reason3,crossindustry_reason4,crossindustry_reason5,inputprovidedfirstname,inputprovidedlastname,inputprovidedstreetaddress,inputprovidedcity,inputprovidedstate,inputprovidedzipcode,inputprovidedssn,inputprovideddateofbirth,inputprovidedphone,inputprovidedlexid,subjectrecordtimeoldest,subjectrecordtimenewest,subjectnewestrecord12month,subjectactivityindex03month,subjectactivityindex06month,subjectactivityindex12month,subjectage,subjectdeceased,subjectssncount,subjectstabilityindex,subjectstabilityprimaryfactor,subjectabilityindex,subjectabilityprimaryfactor,subjectwillingnessindex,subjectwillingnessprimaryfactor,confirmationsubjectfound,confirmationinputname,confirmationinputdob,confirmationinputssn,confirmationinputaddress,sourcenonderogprofileindex,sourcenonderogcount,sourcenonderogcount03month,sourcenonderogcount06month,sourcenonderogcount12month,sourcecredheadertimeoldest,sourcecredheadertimenewest,sourcevoterregistration,educationattendance,educationevidence,educationprogramattended,educationinstitutionprivate,educationinstitutionrating,profliccount,proflictypecategory,businessassociation,businessassociationindex,businessassociationtimeoldest,businesstitleleadership,assetindex,assetindexprimaryfactor,assetownership,assetprop,assetpropindex,assetpropevercount,assetpropcurrentcount,assetpropcurrenttaxtotal,assetproppurchasecount12month,assetproppurchasetimeoldest,assetproppurchasetimenewest,assetpropnewestmortgagetype,assetpropeversoldcount,assetpropsoldcount12month,assetpropsaletimeoldest,assetpropsaletimenewest,assetpropnewestsaleprice,assetpropsalepurchaseratio,assetpersonal,assetpersonalcount,purchaseactivityindex,purchaseactivitycount,purchaseactivitydollartotal,derogseverityindex,derogcount,derogcount12month,derogtimenewest,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount,criminalnonfelonycount12month,criminalnonfelonytimenewest,evictioncount,evictioncount12month,evictiontimenewest,lienjudgmentseverityindex,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentsmallclaimscount,lienjudgmentcourtcount,lienjudgmenttaxcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmenttimenewest,lienjudgmentdollartotal,bankruptcycount,bankruptcycount24month,bankruptcytimenewest,bankruptcychapter,bankruptcystatus,bankruptcydismissed24month,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,inquiryauto12month,inquirybanking12month,inquirytelcom12month,inquirynonshortterm12month,inquiryshortterm12month,inquirycollections12month,ssnsubjectcount,ssndeceased,ssndatelowissued,ssnproblems,addronfilecount,addronfilecorrectional,addronfilecollege,addronfilehighrisk,addrinputtimeoldest,addrinputtimenewest,addrinputlengthofres,addrinputsubjectcount,addrinputmatchindex,addrinputsubjectowned,addrinputdeedmailing,addrinputownershipindex,addrinputphoneservice,addrinputphonecount,addrinputdwelltype,addrinputdwelltypeindex,addrinputdelivery,addrinputtimelastsale,addrinputlastsaleprice,addrinputtaxvalue,addrinputtaxyr,addrinputtaxmarketvalue,addrinputavmvalue,addrinputavmvalue12month,addrinputavmratio12monthprior,addrinputavmvalue60month,addrinputavmratio60monthprior,addrinputcountyratio,addrinputtractratio,addrinputblockratio,addrinputproblems,addrcurrenttimeoldest,addrcurrenttimenewest,add

In [11]:
df['applicant_age'].describe()

count    278037.000000
mean         42.625361
std          14.969870
min           0.000000
25%          29.924875
50%          42.361947
75%          52.077861
max         136.232330
Name: applicant_age, dtype: float64

In [12]:
def get_age(app_date, dob):
#     app_date= app_date.copy()
#     dob= dob.copy()
    
    if (pd.isna(app_date)== False) and (pd.isna(dob)== False):
        
        app_date= pd.to_datetime(app_date)
        dob= pd.to_datetime(dob)
        
        if dob >= app_date:
            dob= dob.replace(year= dob.year - 100)
        else:
            dob= dob

        app_year, app_month, app_day= app_date.year, app_date.month, app_date.day
        dob_year, dob_month, dob_day= dob.year, dob.month, dob.day
        
        age= (app_year - dob_year) + ((app_month - dob_month)/12) + ((app_day - dob_day)/365.5)
        
        return age
    
    else:
        return np.NaN

In [13]:
df[df['applicant_age']<0].shape

(0, 342)

In [15]:
df[df['applicant_age']<0][['client', 'applicant_age', 'applicant_dob', 'app_date']].head()

,client,applicant_age,applicant_dob,app_date


In [16]:
# df['applicant_age']= df.apply(lambda x: get_age(x['app_date'], x['applicant_dob']) if x['applicant_age']< 0 else x['applicant_age'], axis= 1)
# df['applicant_age'].describe()

In [17]:
# df.loc[732066][['client', 'applicant_age', 'applicant_dob', 'app_date']]

In [18]:
# df.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_All_Updated_Data_090424.csv', index= False)

In [19]:
# del df

In [20]:
df= pd.read_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_All_Updated_Data_090424.csv')
df.shape

(290379, 342)

In [21]:
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedGroupKFold, StratifiedKFold

In [22]:
# sgkf= StratifiedGroupKFold(n_splits= 5, shuffle= True, random_state= 127)

In [28]:
T1,T2= train_test_split(df, test_size= 0.20, random_state= 2024)
T1.shape, T2.shape

((232303, 342), (58076, 342))

In [29]:
T1[['approve_flag', 'funded_flag', 'bad_flag']].sum(), T2[['approve_flag', 'funded_flag', 'bad_flag']].sum()

(approve_flag    123785.0
 funded_flag     113618.0
 bad_flag          3800.0
 dtype: float64,
 approve_flag    31097.0
 funded_flag     28462.0
 bad_flag          963.0
 dtype: float64)

In [30]:
T1['bad_flag'].sum()/T1['funded_flag'].sum(), T2['bad_flag'].sum()/T2['funded_flag'].sum()

(0.03344540477741203, 0.03383458646616541)

In [31]:
T1['approve_flag'].mean(), T2['approve_flag'].mean()

(0.532860100816606, 0.5354535436324815)

In [32]:
T1.groupby('client').agg(
    ttd= ('unique_id', 'count'),
    approved= ('approve_flag', 'sum'),
    funded= ('funded_flag', 'sum'),
    bad= ('bad_flag', 'sum'),
    avg_fico= ('fico_pb', 'mean'),
    avg_auto_score= ('auto_score', 'mean'),
    min_app_date=  ('app_date', 'min'),
    max_app_date= ('app_date', 'max')
)

,ttd,approved,funded,bad,avg_fico,avg_auto_score,min_app_date,max_app_date
client,,,,,,,,
Arkansas,31930,9446.0,8454.0,233.0,560.225525,709.901472,2020-01-01,2022-09-29
Bethpage,49042,17215.0,13642.0,554.0,662.564224,713.965173,2017-01-01,2019-06-30
CPM,22189,20133.0,19096.0,465.0,566.505295,718.990896,2016-12-09,2022-09-29
GESA,7378,7378.0,7378.0,179.0,584.301374,714.848197,2018-01-02,2019-09-14
Guardian,40918,18330.0,17543.0,577.0,585.643956,701.985556,2019-11-05,2022-09-29
HFCU,3807,2454.0,2082.0,55.0,622.512746,721.180982,2018-11-27,2020-05-16
Hawaii,8972,5650.0,5610.0,60.0,597.752118,708.088832,2019-01-02,2022-09-29
Numerica,36984,23540.0,23539.0,1076.0,574.187412,703.037584,2017-01-01,2020-10-31
Tower,31083,19639.0,16274.0,601.0,639.812534,730.548113,2017-01-01,2022-05-20


In [33]:
T2.groupby('client').agg(
    ttd= ('unique_id', 'count'),
    approved= ('approve_flag', 'sum'),
    funded= ('funded_flag', 'sum'),
    bad= ('bad_flag', 'sum'),
    avg_fico= ('fico_pb', 'mean'),
    avg_auto_score= ('auto_score', 'mean'),
    min_app_date=  ('app_date', 'min'),
    max_app_date= ('app_date', 'max')
)

,ttd,approved,funded,bad,avg_fico,avg_auto_score,min_app_date,max_app_date
client,,,,,,,,
Arkansas,7858,2331.0,2084.0,62.0,559.198906,709.245737,2020-01-01,2022-09-29
Bethpage,12401,4442.0,3520.0,130.0,662.986425,713.555117,2017-01-01,2019-06-30
CPM,5553,5057.0,4801.0,116.0,558.428957,718.951378,2016-12-28,2022-09-29
GESA,1829,1829.0,1829.0,33.0,590.979086,714.227447,2018-01-02,2019-09-14
Guardian,10283,4574.0,4372.0,166.0,583.877639,701.094817,2019-11-05,2022-09-29
HFCU,1014,643.0,533.0,18.0,617.796020,721.422091,2018-11-27,2020-05-14
Hawaii,2238,1421.0,1409.0,17.0,604.623324,707.869526,2019-01-02,2022-09-29
Numerica,9191,5841.0,5841.0,262.0,573.394886,702.595474,2017-01-03,2020-10-31
Tower,7709,4959.0,4073.0,159.0,640.340771,730.509923,2017-01-01,2022-05-20


In [34]:
# T2.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_OOT_Data_090424.csv', index= False)
# T1.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_DEV_Data_090424.csv', index= False)

In [35]:
del df, T2

### Now Splitting the data into Train and Test

In [36]:
T1.shape

(232303, 342)

In [37]:
T1['Borrower_Race_Prim_BISG'].value_counts(dropna= False)

Borrower_Race_Prim_BISG
white       92628
missing     90433
black       25321
hispanic    20812
api          2881
multiple      122
aian          100
others          6
Name: count, dtype: int64

In [38]:
T1['borrower_gender'].value_counts(dropna= False)

borrower_gender
M    109297
F    101574
I     21432
Name: count, dtype: int64

In [39]:
T1['Race']= T1['Borrower_Race_Prim_BISG']
T1['Gender']= T1['borrower_gender']
T1['age_cuts']= np.where(T1['applicant_age'].isnull(), 'missing',
                         np.where(T1['applicant_age']< 62, 'Age below 62',
                                  np.where(T1['applicant_age']>= 62, 'Age 62 and over', 'missing')))

T1['age_cuts'].value_counts(dropna= False)

age_cuts
Age below 62       197079
Age 62 and over     25343
missing              9881
Name: count, dtype: int64

In [40]:
T1.groupby('age_cuts')['applicant_age'].agg(['min', 'max'])

,min,max
age_cuts,,
Age 62 and over,62.0,136.232330
Age below 62,0.0,61.998746
missing,NaN,NaN


In [47]:
T3,T4= train_test_split(T1, test_size= 0.3, random_state= 2022)
T3.shape, T4.shape

((162612, 345), (69691, 345))

In [48]:
T3[['approve_flag', 'funded_flag', 'bad_flag']].sum(), T4[['approve_flag', 'funded_flag', 'bad_flag']].sum()

(approve_flag    86659.0
 funded_flag     79589.0
 bad_flag         2652.0
 dtype: float64,
 approve_flag    37126.0
 funded_flag     34029.0
 bad_flag         1148.0
 dtype: float64)

In [49]:
T3['approve_flag'].mean(), T4['approve_flag'].mean()

(0.5329188497773842, 0.5327230201891205)

In [50]:
# T3.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_Train_RI_Data_090424.csv', index= False)
# T4.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_Test_RI_Data_090424.csv', index= False)

### Next Part Funded

In [4]:
Train = pd.read_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_Train_RI_Data_090424.csv')
Test = pd.read_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_Test_RI_Data_090424.csv')
Oot= pd.read_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_OOT_Data_090424.csv')

Train.shape, Test.shape, Oot.shape

((162612, 345), (69691, 345), (58076, 342))

In [5]:
Oot['Race']= Oot['Borrower_Race_Prim_BISG']
Oot['Gender']= Oot['borrower_gender']
Oot['age_cuts']= np.where(Oot['applicant_age'].isnull(), 'missing',
                         np.where(Oot['applicant_age']< 62, 'Age below 62',
                                  np.where(Oot['applicant_age']>= 62, 'Age 62 and over', 'missing')))

Oot['age_cuts'].value_counts(dropna= False)

age_cuts
Age below 62       49425
Age 62 and over     6190
missing             2461
Name: count, dtype: int64

In [6]:
TTD= pd.concat([Train, Test]).reset_index(drop= True)
TTD.shape

(232303, 345)

In [7]:
funded_dev= TTD[TTD['funded_flag']== 1].reset_index(drop= True)
funded_dev.shape

(113618, 345)

In [8]:
scored= pd.read_csv('/root/Unified_LN_V2/LN_Unsec_V2/Model_Data/RI_Unsec_Model/090424_RI_Auto_Model_Scored_Data.csv')
scored.shape

(290379, 11)

In [9]:
scored['unique_id']= scored['unique_id'].astype(str)
funded_dev['unique_id']= funded_dev['unique_id'].astype(str)

In [10]:
funded_dev= funded_dev.merge(scored[['unique_id', 'client', 'p1', 'weight']], on= ['unique_id', 'client'], how= 'left')
funded_dev.shape

(113618, 347)

In [11]:
funded_dev[['p1', 'weight']].isnull().sum()

p1        0
weight    0
dtype: int64

In [12]:
funded_dev.rename(columns= {'p1': 'p1_ri'}, inplace= True)

In [13]:
funded_dev[['p1_ri', 'weight']].describe()

,p1_ri,weight
count,113618.000000,113618.000000
mean,0.603445,1.775880
std,0.135818,0.578609
min,0.086108,1.114797
25%,0.516195,1.421909
50%,0.624048,1.602441
75%,0.703280,1.937252
max,0.897024,11.613323


In [14]:
from sklearn.model_selection import train_test_split

In [20]:
F1, F2= train_test_split(funded_dev, test_size= 0.3, random_state= 91)
print(F1['bad_flag'].mean(), F2['bad_flag'].mean())
print(sum(F1['bad_flag']*F1['weight'])/F1['weight'].sum(), sum(F2['bad_flag']*F2['weight'])/F2['weight'].sum())

0.033521098425790874 0.033268790705861646
0.03978399966452742 0.03991933003260507


In [21]:
F1.shape, F2.shape

((79532, 347), (34086, 347))

In [22]:
# F1.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/2_LN_Unsec_V2_Train_Funded_Data_100424.csv', index= False)
# F2.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/2_LN_Unsec_V2_Test_Funded_Data_100424.csv', index= False)
# Oot[Oot['funded_flag']== 1].to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/2_LN_Unsec_V2_Oot_Funded_Data_100424.csv', index= False)

In [23]:
Oot[Oot['funded_flag']== 1].shape

(28462, 345)

In [24]:
# TTD.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/2_LN_Unsec_V2_TTD_Data_100424.csv', index= False)